In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import pickle
import os

from tensorflow.keras import layers, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# pip install -r requirements.txt

In [ ]:
# pip install tensorflow

In [ ]:
# pip install transformers

In [ ]:
# pip install torch

In [ ]:
# Globals for automatic nature
MAKE_DATA = True
ROWS = 500

MAKE_PREPROCESS = True
LOAD_PREPROCESS = False

In [ ]:
# Load data
from data.data import Data

if MAKE_DATA:
    data_raw = Data().get_all_data()
    df = data_raw['MBTI 500']

    df = df[['type', 'posts']]
    df.rename(columns={'posts': 'text'}, inplace=True)
    df = df.sample(n = ROWS, ignore_index = True)

    print(f"MBTI_500 loaded with shape: {df.shape}")


In [ ]:
# Bert Preprocessing
from scripts.BERT_preprocessing import bert_training_preprocessing

if MAKE_PREPROCESS:
    df_list = bert_training_preprocessing(df)
    with open(f'data/prepro/BERT_pp_{ROWS}.pkl', 'wb') as file:
            pickle.dump(df_list, file)
            
    with open(f'data/prepro/BERT_pp_{ROWS}.pkl', 'rb') as file:
            df_list = pickle.load(file)
            
    print(f"Made BERT PrePro set of size {df_list[0]}")
            
if LOAD_PREPROCESS:
    with open(f'data/prepro/BERT_pp_{ROWS}.pkl', 'rb') as file:
            df_list = pickle.load(file)
    print(f"Loaded BERT PrePro set of size {df_list[0].shape}")

In [ ]:
Xlist = []
for i in range(len(df_list[0]['embeddings'])):
    z = np.array(df_list[0]['embeddings'][i])
    z_pad = np.pad(z, ((0, 400 - z.shape[0]), (0, 0)), 'constant', constant_values=-1000)
    Xlist.append(z_pad)
X_ug = np.vstack(Xlist)

In [ ]:
y = df_list[0].iloc[:,[0]]
X = X_ug.reshape(y.shape[0],-1,768)
print(y.shape, X.shape)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()
ylabels = label_as_binary.fit_transform(y)

In [66]:
#0. Sequential Model
model = Sequential()

model.add(layers.Masking(mask_value=-1000))

#1. Normalization
model.add(layers.Normalization())

#2. SimpleRNN layer, 20 units, tanh
model.add(layers.LSTM(units = 25, activation = 'softmax'))


model.add(layers.Dense(512, activation='sigmoid', kernel_regularizer='l1'))

model.add(layers.Dropout(.30))

model.add(layers.Dense(512//2, activation='sigmoid', kernel_regularizer='l1'))
model.add(layers.Dense(512//4, activation='sigmoid', kernel_regularizer='l1'))
model.add(layers.Dense(512//8, activation='sigmoid', kernel_regularizer='l1'))

model.add(layers.Dropout(.50))

model.add(layers.Dense(32, activation='softmax', kernel_regularizer='l2'))


#4 Dense Layer with Return Layer
model.add(layers.Dense(1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', 
              optimizer='Adam',
              metrics = ['mse'])

In [67]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 5)

history = model.fit(X, ylabels,
                  validation_split = .5,
                  epochs = 50,
                  batch_size = 32,
                  callbacks = [es])

Epoch 1/50
23/23 [==============================] - 8s 275ms/step - loss: 75.4270 - mse: 0.2397 - val_loss: 56.4744 - val_mse: 0.2788
Epoch 2/50
23/23 [==============================] - 6s 257ms/step - loss: 42.9848 - mse: 0.2082 - val_loss: 29.4703 - val_mse: 0.3169
Epoch 3/50
23/23 [==============================] - 6s 248ms/step - loss: 20.4082 - mse: 0.1778 - val_loss: 12.4094 - val_mse: 0.3547
Epoch 4/50
23/23 [==============================] - 6s 256ms/step - loss: 7.8817 - mse: 0.1548 - val_loss: 5.2476 - val_mse: 0.3857
Epoch 5/50
23/23 [==============================] - 6s 250ms/step - loss: 3.2691 - mse: 0.1384 - val_loss: 2.6341 - val_mse: 0.4107
Epoch 6/50
23/23 [==============================] - 6s 249ms/step - loss: 1.5887 - mse: 0.1257 - val_loss: 1.8256 - val_mse: 0.4330
Epoch 7/50
23/23 [==============================] - 6s 255ms/step - loss: 0.9732 - mse: 0.1147 - val_loss: 1.5108 - val_mse: 0.4525
Epoch 8/50
23/23 [==============================] - 6s 248ms/step - lo

In [ ]:
model.summary()

In [ ]:
pd.DataFrame(history.history)

# Model Tuning
## n = 500
loss: 0.5602 - val_loss: 1.1611, opt = Adam



In [ ]:
# Create or Load Models

DIE

from model.DL import *

model_list = initialize_models(metrics='accuracy',
                               loss='binary_crossentropy',
                               vocab_size=768)

In [ ]:
model_list[0].summary()

In [ ]:
# Train Model

train_models(df_list, model_list)

In [ ]:
# Predict Model